# Airbnb Profitability Function

The purpose is to create a function to estimate if it is profitable for a existing or perspective homeowner to invest in a property to Airbnb. 

Effectively, the forumla will take this form:

        Profit = Airbnb Revenue - Mortgage Payments

Airbnb revenue and the cost of the mortgage will be calculated over a 12 month period, so profit will represent yearly profit.

Average mortgage rate for a 30-year mortgage for the state of Washington:  6.20% (as of Wednesday, September 14th, 2022)
[Source](https://www.bankrate.com/mortgages/mortgage-rates/washington/?mortgageType=Refinance&partnerId=Refinance&pid=br3&pointsChanged=false&refinanceCashOutAmount=0&refinanceLoanAmount=765600&refinanceLoanTerms=30yr%2C15yr&refinancePoints=All&refinancePropertyType=SingleFamily&refinancePropertyUse=PrimaryResidence&refinancePropertyValue=957000&searchChanged=false&showingStacked=all&ttcid&userCreditScore=740&userFha=false&userVeteranStatus=NoMilitaryService&zipCode=10270)

Average Airbnb Occupancy Rate for King County: 87.91% 
    [Source](https://www.dpgo.com/go/top-10-vacation-rental-locations/)

In [2]:
avg_mortgage_rate = 0.0620
avg_airbnb_occupancy = .59

#.8791

Load in Airbnb Average Price and House Price data based on Seattle zipcode

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from   sklearn.linear_model import LinearRegression
from   sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

In [4]:
df = pd.read_excel('/Users/weatherford/Documents/Flatiron/Phase_2_Housing_Project/data/king_county_home_sales.xlsx')

In [5]:
df

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,sqft_lot15,renovated,view_ord,condition_ord,grade_ord,age,point,distance_to_starbucks_miles,distance_to_amazon_miles,distance_to_needle_miles
0,0,7129300520,2014-10-13,221900,3,1.00,1180,5650,1.0,0,...,5650,0,0,2,4,60,"(47.5112, -122.257)",6.041009,8.552873,8.697321
1,1,6414100192,2014-12-09,538000,3,2.25,2570,7242,2.0,0,...,7639,1,0,2,4,64,"(47.5112, -122.257)",6.041009,8.552873,8.697321
2,2,5631500400,2015-02-25,180000,2,1.00,770,10000,1.0,0,...,8062,0,0,2,3,82,"(47.721, -122.319)",9.717661,6.846950,7.085697
3,3,2487200875,2014-12-09,604000,4,3.00,1960,5000,1.0,0,...,5000,0,0,4,4,50,"(47.7379, -122.233)",11.858223,9.319655,9.758480
4,4,1954400510,2015-02-18,510000,3,2.00,1680,8080,1.0,0,...,7503,0,0,2,5,28,"(47.5208, -122.393)",4.945854,7.509964,7.184440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,21592,263000018,2014-05-21,360000,3,2.50,1530,1131,3.0,0,...,1509,0,0,2,5,6,"(47.5773, -122.409)",3.456165,4.604793,4.085285
21593,21593,6600060120,2015-02-23,400000,4,2.50,2310,5813,2.0,0,...,7200,0,0,2,5,1,"(47.6993, -122.346)",8.204234,5.315147,5.446183
21594,21594,1523300141,2014-06-23,402101,2,0.75,1020,1350,2.0,0,...,2007,0,0,2,4,6,"(47.5107, -122.362)",4.999896,7.821711,7.608764
21595,21595,291310100,2015-01-16,400000,3,2.50,1600,2388,2.0,0,...,1287,0,0,2,5,11,"(47.5944, -122.299)",1.938102,2.628624,2.962102


In [6]:
len(df['zipcode'].unique())

70

In [7]:
seattle_df = pd.read_csv('/Users/weatherford/Documents/Flatiron/Phase_2_Housing_Project/data/seattle.csv')
seattle_df = seattle_df.drop(columns= 'Unnamed: 0')
seattle_df

,id_x,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,sqft_basement,...,renov_yr_clean,view_ord,condition_ord,grade_ord,age,id_y,latitude,longitude,airbnb_avg_price,house_price
0,7129300520,2014-10-13,3,1.00,1180,5650,1.0,0,1180,0.0,...,1955.0,0.0,2.0,4.0,60,7.345680e+16,47.500403,-122.243751,170.121951,221900.0
1,4060000240,2014-06-23,2,1.00,880,6780,1.0,0,880,0.0,...,1945.0,0.0,3.0,3.0,70,7.345680e+16,47.500403,-122.243751,170.121951,205425.0
2,4058801670,2014-07-17,3,2.25,2100,8201,1.0,0,1620,480.0,...,1967.0,2.0,2.0,5.0,48,7.345680e+16,47.500403,-122.243751,170.121951,445000.0
3,2976800796,2014-09-25,3,1.00,1300,5898,1.0,0,1300,0.0,...,1961.0,0.0,2.0,4.0,54,7.345680e+16,47.500403,-122.243751,170.121951,236000.0
4,6874200960,2015-02-27,2,1.00,860,5265,1.0,0,860,0.0,...,1931.0,0.0,2.0,3.0,84,7.345680e+16,47.500403,-122.243751,170.121951,170000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9351,2895800590,2014-10-20,5,2.50,2170,2752,2.0,0,2170,0.0,...,2014.0,0.0,2.0,5.0,1,6.652046e+16,47.541882,-122.358118,176.106557,359800.0
9352,3438503021,2014-11-05,3,2.50,2430,7049,2.0,0,2430,0.0,...,2007.0,0.0,2.0,5.0,8,6.652046e+16,47.541882,-122.358118,176.106557,443000.0
9353,880000205,2014-07-29,3,2.00,1260,1125,2.0,0,810,450.0,...,2011.0,0.0,2.0,4.0,4,6.652046e+16,47.541882,-122.358118,176.106557,249000.0
9354,3438501329,2014-05-20,2,2.50,1590,2656,2.0,0,1220,370.0,...,2009.0,0.0,2.0,4.0,6,6.652046e+16,47.541882,-122.358118,176.106557,305000.0


Create a correlation heatmap based on the primary drivers of House Sale Price

In [11]:
seattle_df.columns

Index(['id_x', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'renov_yr_clean', 'view_ord', 'condition_ord', 'grade_ord', 'age',
       'id_y', 'latitude', 'longitude', 'airbnb_avg_price', 'house_price'],
      dtype='object')

In [10]:
#first create a subset of the seattle_df with the highest correlated variables
seattle_corr = seattle_df.drop(columns= ['id_x','date',
       'floors', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_lot15',
       'renov_yr_clean', 'condition_ord', 'age',
       'id_y', 'latitude', 'longitude','airbnb_avg_price',
       'profit', 'profitable?'])
seattle_corr

KeyError: "['profit', 'profitable?'] not found in axis"

In [ ]:
corr = seattle_corr.corr()[['house_price']].sort_values(by='house_price', ascending=False)
corr = corr.drop('house_price')

sns.set(rc={'figure.figsize':(11.7,8.27)})
heatmap = sns.heatmap(corr, annot=True)

heatmap.set_title('Feature Correlation with Sale Price', fontdict={'fontsize':18}, pad=16)
heatmap.set_xlabel('Sale Price', fontdict={'fontsize':16})
heatmap.set_ylabel('Highest Correlated Features', fontdict={'fontsize':16})
heatmap.set(xticklabels=[])
plt.savefig('price_correlation_heatmap.png');

In [ ]:
corr = seattle_corr.corr()[['house_price']].sort_values(by='house_price', ascending=False)

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(corr, annot=True)

In [12]:
len(seattle_df.zipcode.unique())

28

In [13]:
airbnb_revenue = seattle_df['airbnb_avg_price'] * (365) * avg_airbnb_occupancy
airbnb_revenue


0       36635.762195
1       36635.762195
2       36635.762195
3       36635.762195
4       36635.762195
            ...     
9351    37924.547131
9352    37924.547131
9353    37924.547131
9354    37924.547131
9355    37924.547131
Name: airbnb_avg_price, Length: 9356, dtype: float64

Mortgage Cost

$$Mortgage Cost = P  \frac{r(1+ r)^{n}}{ (1+r)^{n} - 1}$$

Where P equals housing price, r equals the interest rate, and n equals the number of payments over the duration of the loan


For our particular model:
- r = 6.20%
- n = 30, as we are calculating yearly mortgage cost for a 30 year mortgage
- P is each house's sale price from the Seattle subset of the King County dataset


In [14]:
seattle_df.columns

Index(['id_x', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'renov_yr_clean', 'view_ord', 'condition_ord', 'grade_ord', 'age',
       'id_y', 'latitude', 'longitude', 'airbnb_avg_price', 'house_price'],
      dtype='object')

In [15]:
len(seattle_df['zipcode'].unique())

28

In [16]:
360000 * (avg_mortgage_rate * (1+avg_mortgage_rate)**30)/(((1+0.062)**30-1)) #Test of the formula

26715.736475425732

In [17]:
annual_mortgage = seattle_df['house_price'] * (avg_mortgage_rate * (1+avg_mortgage_rate)**30)/((1+0.062)**30-1)
annual_mortgage

0       16467.283122
1       15244.667126
2       33023.618699
3       17513.649467
4       12615.764447
            ...     
9351    26700.894400
9352    32875.197941
9353    18478.384396
9354    22634.165625
9355    24489.425102
Name: house_price, Length: 9356, dtype: float64

In [18]:
profit = airbnb_revenue - annual_mortgage
profit
profit.describe()

count      9356.000000
mean      -2139.642245
std       26491.667954
min     -526515.970671
25%       -8916.553928
50%        2330.446500
75%       11116.147606
max       71036.089344
dtype: float64

In [19]:
seattle_df['profit'] = profit
seattle_df

,id_x,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,sqft_basement,...,view_ord,condition_ord,grade_ord,age,id_y,latitude,longitude,airbnb_avg_price,house_price,profit
0,7129300520,2014-10-13,3,1.00,1180,5650,1.0,0,1180,0.0,...,0.0,2.0,4.0,60,7.345680e+16,47.500403,-122.243751,170.121951,221900.0,20168.479073
1,4060000240,2014-06-23,2,1.00,880,6780,1.0,0,880,0.0,...,0.0,3.0,3.0,70,7.345680e+16,47.500403,-122.243751,170.121951,205425.0,21391.095069
2,4058801670,2014-07-17,3,2.25,2100,8201,1.0,0,1620,480.0,...,2.0,2.0,5.0,48,7.345680e+16,47.500403,-122.243751,170.121951,445000.0,3612.143496
3,2976800796,2014-09-25,3,1.00,1300,5898,1.0,0,1300,0.0,...,0.0,2.0,4.0,54,7.345680e+16,47.500403,-122.243751,170.121951,236000.0,19122.112728
4,6874200960,2015-02-27,2,1.00,860,5265,1.0,0,860,0.0,...,0.0,2.0,3.0,84,7.345680e+16,47.500403,-122.243751,170.121951,170000.0,24019.997748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9351,2895800590,2014-10-20,5,2.50,2170,2752,2.0,0,2170,0.0,...,0.0,2.0,5.0,1,6.652046e+16,47.541882,-122.358118,176.106557,359800.0,11223.652732
9352,3438503021,2014-11-05,3,2.50,2430,7049,2.0,0,2430,0.0,...,0.0,2.0,5.0,8,6.652046e+16,47.541882,-122.358118,176.106557,443000.0,5049.349191
9353,880000205,2014-07-29,3,2.00,1260,1125,2.0,0,810,450.0,...,0.0,2.0,4.0,4,6.652046e+16,47.541882,-122.358118,176.106557,249000.0,19446.162736
9354,3438501329,2014-05-20,2,2.50,1590,2656,2.0,0,1220,370.0,...,0.0,2.0,4.0,6,6.652046e+16,47.541882,-122.358118,176.106557,305000.0,15290.381506


In [20]:
seattle_df["profitable?"]= np.where(seattle_df['profit']>0, 1, 0)
seattle_df

,id_x,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,sqft_basement,...,condition_ord,grade_ord,age,id_y,latitude,longitude,airbnb_avg_price,house_price,profit,profitable?
0,7129300520,2014-10-13,3,1.00,1180,5650,1.0,0,1180,0.0,...,2.0,4.0,60,7.345680e+16,47.500403,-122.243751,170.121951,221900.0,20168.479073,1
1,4060000240,2014-06-23,2,1.00,880,6780,1.0,0,880,0.0,...,3.0,3.0,70,7.345680e+16,47.500403,-122.243751,170.121951,205425.0,21391.095069,1
2,4058801670,2014-07-17,3,2.25,2100,8201,1.0,0,1620,480.0,...,2.0,5.0,48,7.345680e+16,47.500403,-122.243751,170.121951,445000.0,3612.143496,1
3,2976800796,2014-09-25,3,1.00,1300,5898,1.0,0,1300,0.0,...,2.0,4.0,54,7.345680e+16,47.500403,-122.243751,170.121951,236000.0,19122.112728,1
4,6874200960,2015-02-27,2,1.00,860,5265,1.0,0,860,0.0,...,2.0,3.0,84,7.345680e+16,47.500403,-122.243751,170.121951,170000.0,24019.997748,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9351,2895800590,2014-10-20,5,2.50,2170,2752,2.0,0,2170,0.0,...,2.0,5.0,1,6.652046e+16,47.541882,-122.358118,176.106557,359800.0,11223.652732,1
9352,3438503021,2014-11-05,3,2.50,2430,7049,2.0,0,2430,0.0,...,2.0,5.0,8,6.652046e+16,47.541882,-122.358118,176.106557,443000.0,5049.349191,1
9353,880000205,2014-07-29,3,2.00,1260,1125,2.0,0,810,450.0,...,2.0,4.0,4,6.652046e+16,47.541882,-122.358118,176.106557,249000.0,19446.162736,1
9354,3438501329,2014-05-20,2,2.50,1590,2656,2.0,0,1220,370.0,...,2.0,4.0,6,6.652046e+16,47.541882,-122.358118,176.106557,305000.0,15290.381506,1


In [21]:
seattle_df.loc[seattle_df.profit < 0]

,id_x,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,sqft_above,sqft_basement,...,condition_ord,grade_ord,age,id_y,latitude,longitude,airbnb_avg_price,house_price,profit,profitable?
18,7567600045,2014-08-27,2,1.00,1150,12775,1.0,1,1150,0.0,...,3.0,3.0,107,7.345680e+16,47.500403,-122.243751,170.121951,825000.0,-24587.800561,0
21,8069000075,2014-12-29,4,1.75,2460,10061,1.0,1,1410,1050.0,...,2.0,4.0,54,7.345680e+16,47.500403,-122.243751,170.121951,790000.0,-21990.437293,0
24,8073000550,2015-04-15,4,3.75,3190,17186,2.0,1,3190,0.0,...,2.0,7.0,16,7.345680e+16,47.500403,-122.243751,170.121951,1700000.0,-89521.882272,0
25,4058800830,2015-03-18,6,3.00,3840,14040,1.5,0,2460,1380.0,...,2.0,5.0,66,7.345680e+16,47.500403,-122.243751,170.121951,612000.0,-8780.989813,0
46,8073000495,2014-10-10,2,1.00,1160,17635,1.0,1,1160,0.0,...,2.0,3.0,70,7.345680e+16,47.500403,-122.243751,170.121951,700000.0,-15311.503174,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9321,3438500036,2015-04-29,5,3.75,2380,7268,1.0,0,1430,950.0,...,2.0,5.0,7,6.652046e+16,47.541882,-122.358118,176.106557,545000.0,-2520.109477,0
9328,3438500253,2014-09-04,5,3.50,3560,5008,2.0,0,2810,750.0,...,2.0,5.0,2,6.652046e+16,47.541882,-122.358118,176.106557,616950.0,-7859.546254,0
9334,2135200155,2014-08-05,5,3.25,3030,7410,2.0,0,2150,880.0,...,2.0,5.0,1,6.652046e+16,47.541882,-122.358118,176.106557,580000.0,-5117.472746,0
9339,3438500250,2014-06-23,5,3.25,2910,5027,2.0,0,2040,0.0,...,2.0,5.0,2,6.652046e+16,47.541882,-122.358118,176.106557,515000.0,-293.798105,0


## EDA for profit to see what the primary drivers are

Hypothesis: probably largely linked to the drivers of price

In [ ]:
plt.figure(figsize=(18,7))
mask = np.triu(np.ones_like(seattle_df.corr(), dtype=bool))
heatmap = sns.heatmap(seattle_df.corr(), annot=True, mask=mask, vmin=-1, vmax=1)
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16)
plt.show();

In [ ]:
corr = seattle_df.corr()[['profit']].sort_values(by='profit', ascending=False)

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.heatmap(corr, annot=True)

# Visualizations for Square Foot of Living Area's effect on profit and house price

Since profit is depending on house price (the more a house's mortgage costs, the less profitable renting it out will be for a set revenue), square foot of the living area is positively correlated with price and negatively correlated with profit.

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data= seattle_df, x="sqft_living", y="house_price", ax=ax, color = "blue")
ax.set_xlim(0, 5000)
ax.set_ylim(0, 3500000)
ax.set_title('House Sale Price vs Square Foot of Living Area', fontdict={'fontsize':18}, pad=16)
ax.set_xlabel('Square Foot of Living Area', fontdict={'fontsize':16})
ax.set_ylabel('Sale Price', fontdict={'fontsize':16})
plt.show()
fig.savefig('sale_price_vs_sqft_living.png');

In [ ]:
seattle_df.columns

In [ ]:
#most profitable zip code
kenmore = seattle_df.loc[seattle_df['zipcode'] == 98028]
print("avgerage price per home for 98028: ", kenmore['house_price'].mean())

#2nd most profitable zip code
df1 = seattle_df.loc[seattle_df['zipcode'] == 98106]
print("avgerage price per home for 98106: ", df1['house_price'].mean())

#3rd most profitable zip code
df2 = seattle_df.loc[seattle_df['zipcode'] == 98168]
print("avgerage price per home for 98168: ", df2['house_price'].mean())

In [ ]:
kenmore.sort_values(by=['profit'], ascending = False)

In [ ]:
kenmore.columns

In [ ]:
sns.set_palette("mako")

In [ ]:
df1.sort_values(by=['profit'], ascending = False)

In [ ]:
df2.sort_values(by=['profit'], ascending = False)

In [ ]:
subset = seattle_df.loc[seattle_df['airbnb_avg_price'] > 300][0:25]
subset['zipcode'].unique()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data= seattle_df, x="sqft_living", y="profit", ax=ax, color = 'red', hue = 'airbnb_avg_price')
ax.set_xlim(0, 6000)
ax.set_ylim(-200000, 150000)
ax.set_title('Airbnb Profit vs Square Foot of Living Area', fontdict={'fontsize':18}, pad=16)
ax.set_xlabel('Square Foot of Living Area', fontdict={'fontsize':16})
ax.set_ylabel('Profit', fontdict={'fontsize':16})

plt.show()

fig.savefig('profit_vs_sqft_living.png');

In [ ]:
print(kenmore['house_price'].max())
print(kenmore['house_price'].min())
kenmore['house_price'].describe()

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data= kenmore, x="sqft_living", y="profit", ax=ax, hue = "house_price", legend = 'auto', palette = "flare")
#ax.set_xlim(0, 6000)
ax.set_ylim(50000, 120000)
ax.set_title('''Airbnb Profit vs Square Foot of Living Area, 
Kenmore Washington''', fontdict={'fontsize':18}, pad=16)
ax.set_xlabel('Square Foot of Living Area', fontdict={'fontsize':16})
ax.set_ylabel('Profit', fontdict={'fontsize':16})
plt.legend(title='House Price', loc='upper right', labels=['250,000', '500,000', '750,000','1,000,000','1,250,000','1,500,000'])

plt.show() 


fig.savefig('sale_price_vs_sqft_living.png');

In [ ]:
X, y = seattle_df[['sqft_living']] , seattle_df[['profit']]

# Generate train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.3, random_state=42
)

# Init, fit, score
multi_model = LinearRegression()
multi_model.fit(X_train, y_train)

print(f"Training score: {multi_model.score(X_train, y_train)}")
multi_model_score = multi_model.score(X_test, y_test)
print(f"Test score: {multi_model.score(X_test, y_test)}")

In [ ]:
df1 = pd.read_excel('/Users/weatherford/Documents/Flatiron/Phase_2_Housing_Project/data/washington_state_income_by_zipcode.xlsx')

In [ ]:
df1['zipcode'] = df1['Zip Code']
df1 = df1.drop(columns=['Zip Code'])
df1

Made dataframe including profitability data and household income data, merged on zipcode, for Seattle metro area

In [ ]:
df = pd.merge(seattle_df, df1, on='zipcode', how='inner')

In [ ]:
df.loc[df.bedrooms == 33]
df = df.drop(labels=3099, axis=0)

In [ ]:
df.columns

In [ ]:
df['Avg. Income/H/hold']

In [ ]:
top_cities = df.groupby(by=['Avg. Income/H/hold']).mean()
top_cities = top_cities.sort_values(by=['profit'], ascending=False)
top_cities

In [ ]:
top_zipcodes = df.groupby(by=['zipcode']).mean()
top_zipcodes = top_zipcodes.sort_values(by=['profit'], ascending=False)
top_zipcodes = top_zipcodes.reset_index()
top_zipcodes

In [ ]:
sns.set_palette(sns.color_palette("hls"))

In [ ]:
plt.figure(figsize=(12,6))
# make barplot and sort bars
sns.barplot(data=top_zipcodes.head(5), x="zipcode", y="profit", order=top_zipcodes.head().sort_values('profit').zipcode)   

#set labels
plt.xlabel("Top Zipcodes", size=18)
plt.ylabel("Avg Profit (Dollars)", size=15)
plt.title("Top 5 Most Profitable Zipcodes for Airbnb", size=18)
plt.tight_layout()
plt.savefig("top_5_protifable_zipcodes.png", dpi=100);



In [ ]:
fig, ax = plt.subplots()
sns.barplot(data= kenmore, x="grade_ord", y="profit", ax=ax, estimator = np.median,ci = 0)
#ax.set_xlim(0, 6000)
ax.set_ylim(50000, 120000)
ax.set_title('''Airbnb Profit vs Square Foot of Living Area, 
Kenmore Washington''', fontdict={'fontsize':18}, pad=16)
ax.set_xlabel('Square Foot of Living Area', fontdict={'fontsize':16})
ax.set_ylabel('Profit', fontdict={'fontsize':16})

plt.show()

#fig.savefig('sale_price_vs_sqft_living.png');

In [ ]:
#df.to_csv('seattle_income_&_profit.csv')

In [ ]:
sns.scatterplot(data= df, x="sqft_living" , y="house_price");

In [ ]:
df['grade_ord'].loc[df['house_price'] >= 5000000]

In [ ]:
sns.barplot(data= df, x="grade_ord" , y="house_price", estimator = np.median,ci = 0)

plt.xlabel("House Grade", size=18)
plt.ylabel("House Price", size=16)
plt.title("House Grade vs Price", size=18)
plt.tight_layout()
plt.savefig("house_grade_barplot.png", dpi=100);

In [ ]:
df.corr()

In [ ]:
df

In [ ]:
sns.scatterplot(data=tips, x="total_bill", y="tip")